In [1]:
import os
import zipfile
import time

import boto3
from sagemaker.tensorflow import TensorFlow

s3 = boto3.client('s3')
sm = boto3.client('sagemaker')

script_s3_bucket = "sagemaker-training-script-experiment"
script_s3_key = "train.zip"
sagemaker_role_arn = "arn:aws:iam::322725876573:role/terraform-20240129003118010100000001"
s3.download_file(script_s3_bucket, script_s3_key, f"/tmp/{script_s3_key}")
with zipfile.ZipFile(f"/tmp/{script_s3_key}", 'r') as f:
    f.extractall('/tmp/')
os.system(f"rm /tmp/{script_s3_key}")
os.system("chmod +x /tmp/train.py")
os.system("mv /tmp/train.py .")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


0

In [2]:
start_time = time.time()
estimator = TensorFlow(
    base_job_name=f"leo-try-training-{int(time.time())}",
    entry_point="train.py",
    # source_dir="/home/ec2-user/SageMaker",
    role=sagemaker_role_arn,
    instance_count=1,
    instance_type="ml.c5.xlarge",
    hyperparameters={"epochs": 10},
    framework_version="2.13.0",
    py_version="py310",
    output_path="s3://sagemaker-training-script-experiment/model",
    # image_name="763104351884.dkr.ecr.eu-west-2.amazonaws.com/tensorflow-training:2.13.0-cpu-py310-ubuntu20.04-ec2",
    script_mode=True,
    input_mode="File"
    # distribution={"parameter_server": {"enabled": True}},
)
estimator.fit("s3://sagemaker-training-script-experiment/data",
              wait=False)
end_time = time.time()
print(f"Training job initialisation takes: {(end_time-start_time)/60} min")

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: leo-try-training-1706624759-2024-01-30-14-25-59-602


Training job initialisation takes: 0.011827623844146729 min
